In [1]:
import os
import glob
import pyart
import netCDF4
import numpy as np
from datetime import datetime

#init vars
grid_folder = '/g/data1a/kl02/jss548/hail-research/processed_data'
prof_folder = '/g/data1a/kl02/jss548/hail-research/profile_data'
hca_hail_idx  = 9

grid_file_list = sorted(glob.glob(grid_folder +'/**/*meshgrids.nc'))
for grid_file in grid_file_list:
    
    #load data
    print('processing: ',grid_file)
    grid    = pyart.io.read_grid(grid_file)
    
    #extract data
    hca  = grid.fields['HCA']['data']
    refl = grid.fields['DBZH_CORR']['data']
    temp = grid.fields['TEMPERATURE']['data']
    zdr  = grid.fields['ZDR_CORR']['data']
    rhv  = grid.fields['RHOHV_CORR']['data']
    hsda = grid.fields['HSDA']['data']
    hdr  = grid.fields['HDR']['data']
    mesh = grid.fields['MESH']['data']

    #extract height layers
    grid_sz    = np.shape(hca)
    prof_z     = grid.z['data']
    
    #extract date
    base_name  = os.path.basename(grid_file)
    fileparts  = base_name.split('_')
    radar_name = fileparts[0]
    radar_dts  = fileparts[1] + '_' + fileparts[2]
    radar_dt   = datetime.strptime(radar_dts, '%Y%m%d_%H%M%S')
    
#     #mask HCA below temp_mask_val
#     temp_mask_val = -5
#     rm_mask       = temp > temp_mask_val
#     hca[rm_mask]  = np.nan

    #mask HCA hail layers
    hca_hail_idx     = hca_hail_idx
    hca_hail_mask    = hca == hca_hail_idx
    #covert to a column mask
    hca_hail_mask_2d = np.max(hca_hail_mask, axis=0)

    if not np.any(hca_hail_mask_2d):
        print('no hail found')
        continue
    
    #extract profiles
    ii, jj = np.where(hca_hail_mask_2d)
    refl_prof     = np.zeros((grid_sz[0], len(ii)))
    temp_prof     = np.zeros((grid_sz[0], len(ii)))
    zdr_prof      = np.zeros((grid_sz[0], len(ii)))
    rhv_prof      = np.zeros((grid_sz[0], len(ii)))
    hca_prof      = np.zeros((grid_sz[0], len(ii)))
    hsda_prof     = np.zeros((grid_sz[0], len(ii)))
    hdr_prof      = np.zeros((grid_sz[0], len(ii)))
    mesh_val      = np.zeros((len(ii)))
    hsda_prof_max = np.zeros((len(ii)))
    prof_dt       = []

    for idx,_ in enumerate(ii):
        refl_prof[:, idx]  = refl[:, ii[idx], jj[idx]]
        temp_prof[:, idx]  = temp[:, ii[idx], jj[idx]]
        zdr_prof[:, idx]   = zdr[:, ii[idx], jj[idx]]
        rhv_prof[:, idx]   = rhv[:, ii[idx], jj[idx]]
        hca_prof[:, idx]   = hca[:, ii[idx], jj[idx]]
        hdr_prof[:, idx]   = hdr[:, ii[idx], jj[idx]]
        mesh_val[idx]      = mesh[0, ii[idx], jj[idx]]
        #hsda
        tmp_prof_hsda      = hsda[:, ii[idx], jj[idx]]
        hsda_prof[:, idx]  = tmp_prof_hsda
        hsda_prof_max[idx] = np.nanmax(tmp_prof_hsda)
        #time
        prof_dt.append(radar_dt)
    
    #remove invalid data
    refl_prof[refl_prof==-9999] = np.nan
    temp_prof[temp_prof==-9999] = np.nan
    zdr_prof[zdr_prof==-9999]   = np.nan
    rhv_prof[rhv_prof==-9999]   = np.nan
    hca_prof[hca_prof>9999]     = np.nan
    hdr_prof[hdr_prof>9999]     = np.nan
    mesh_val[mesh_val==-9999]   = np.nan
    hsda_prof[hsda_prof>3]      = np.nan

    #save data
    out_fn = '_'.join([radar_name, radar_dts, 'hailprofile.nc'])
    out_ffn  = '/'.join([prof_folder, out_fn])
    with netCDF4.Dataset(out_ffn, "w", format="NETCDF4") as rootgrp:
        # Create time dimension
        tddim = len(prof_dt)
        rootgrp.createDimension('profile_time_dim', tddim)
        # Create altitude dimension
        zddim = len(prof_z)
        rootgrp.createDimension('profile_alt_dim', zddim)
                
        #create time vars
        dim_time       = rootgrp.createVariable("profile_time", "f8", ('profile_time_dim'))
        dim_time.units = "seconds since 1970-01-01T00:00:00Z"
        dim_time[:]    = netCDF4.date2num(prof_dt, "seconds since 1970-01-01T00:00:00Z")
        #create alt vars
        dim_alt       = rootgrp.createVariable("profile_alt", "f8", ('profile_alt_dim'))
        dim_alt.units = "Meters above sea level"
        dim_alt[:]    = prof_z
        
        #create refl_prof data
        d1_clutgrp       = rootgrp.createVariable('refl_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d1_clutgrp.units = "Reflectivity (dB)"
        d1_clutgrp.setncattr_string("description", "Reflectivity Profiles")
        d1_clutgrp[:]    = refl_prof
        #create temp_prof data
        d1a_clutgrp       = rootgrp.createVariable('temp_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d1a_clutgrp.units = "Temperature (C)"
        d1a_clutgrp.setncattr_string("description", "Temperature Profiles")
        d1a_clutgrp[:]    = temp_prof
        #create zdr_prof data
        d2_clutgrp       = rootgrp.createVariable('zdr_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d2_clutgrp.units = "Differential Reflectivity (dB)"
        d2_clutgrp.setncattr_string("description", "ZDR Profiles")
        d2_clutgrp[:]    = zdr_prof
        #create rhv_prof data
        d3_clutgrp       = rootgrp.createVariable('rhv_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d3_clutgrp.units = "ratio"
        d3_clutgrp.setncattr_string("description", "RHOHV Profiles")
        d3_clutgrp[:]    = rhv_prof
        #create hdr_prof data
        d4_clutgrp       = rootgrp.createVariable('hdr_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d4_clutgrp.units = "mm"
        d4_clutgrp.setncattr_string("description", "HDR Profiles")
        d4_clutgrp[:]    = hdr_prof
        #create mesh_prof data
        d5_clutgrp       = rootgrp.createVariable('mesh_prof', 'f8', ("profile_time_dim"), zlib=True)
        d5_clutgrp.units = "MESH (mm)"
        d5_clutgrp.setncattr_string("description", "MESH Values")
        d5_clutgrp[:]    = mesh_val
        #create hsda_prof data
        d6_clutgrp       = rootgrp.createVariable('hsda_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d6_clutgrp.units = "1: Small, 2: Large, 3: Giant"
        d6_clutgrp.setncattr_string("description", "HSDA")
        d6_clutgrp[:]    = hsda_prof
        #create hca_prof data
        d7_clutgrp       = rootgrp.createVariable('hca_prof', 'f8', ("profile_alt_dim", "profile_time_dim"), zlib=True)
        d7_clutgrp.units = "class"
        d7_clutgrp.setncattr_string("description", "HCA Profiles")
        d7_clutgrp[:]    = hca_prof
    
print('finished')


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119

processing:  /g/data1a/kl02/jss548/hail-research/processed_data/66_20170922/66_20170922_020028_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/66_20170922/66_20170922_020628_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/66_20170922/66_20170922_021229_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/66_20170922/66_20170922_021828_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/66_20170922/66_20170922_022428_meshgrids.nc
processing:  /

/g/data/en0/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/ipykernel_launcher.py:76: UserWarning: Warning: converting a masked element to nan.
/g/data/en0/jss548/miniconda3/envs/radar/lib/python3.6/site-packages/ipykernel_launcher.py:80: UserWarning: Warning: converting a masked element to nan.


processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_051239_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_051804_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_052404_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_053004_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_053604_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_054204_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_054804_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_055404_meshgrids.nc
processing:  /g/data1a/kl02/jss548/hail-research/processed_data/CP2_20081116/CP2_20081116_060003_meshgrids.nc
processing